In [ ]:
!pip install autogluon.tabular[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 84.0 

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np
import random
from sklearn.datasets import make_regression
from datetime import datetime

x, y = make_regression(n_samples = 100,n_features = 5,n_targets = 1, random_state = 1)
dfx = pd.DataFrame(x, columns=['A','B','C','D','E'])
dfy = pd.DataFrame(y, columns=['label'])

# Create an integer column, a datetime column, a categorical column and a string column to demonstrate how they are processed.
dfx['B'] = (dfx['B']).astype(int)
dfx['C'] = datetime(2000,1,1) + pd.to_timedelta(dfx['C'].astype(int), unit='D')
dfx['D'] = pd.cut(dfx['D'] * 10, [-np.inf,-5,0,5,np.inf],labels=['v','w','x','y'])
dfx['E'] = pd.Series(list(' '.join(random.choice(["abc", "d", "ef", "ghi", "jkl"]) for i in range(4)) for j in range(100)))
dataset=TabularDataset(dfx)
print(dfx)

           A  B          C  D                E
0  -0.545774  0 2000-01-01  y     d ef abc abc
1  -0.468674  0 2000-01-02  x      jkl d jkl d
2   1.767960  0 1999-12-31  v     jkl jkl ef d
3  -0.118771  1 2000-01-01  y  abc ghi jkl jkl
4   0.630196  0 1999-12-31  w     ef d abc jkl
..       ... ..        ... ..              ...
95 -1.182318 -1 2000-01-01  v       d d ghi ef
96  0.562761  0 2000-01-01  v  ghi ghi ghi jkl
97 -0.797270  0 2000-01-01  w      jkl d jkl d
98  0.502741  0 1999-12-31  y  abc abc abc jkl
99  2.056356  0 1999-12-30  w  jkl abc abc ghi

[100 rows x 5 columns]


In [ ]:
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
auto_ml_pipeline_feature_generator.fit_transform(X=dfx)

,A,B,D,E,C,C.year,C.month,C.day,C.dayofweek,E.char_count,E.symbol_ratio.,__nlp__.abc,__nlp__.ef,__nlp__.ghi,__nlp__.jkl,__nlp__._total_
0,-0.545774,0,3,NaN,946684800000000000,2000,1,1,5,4,3,2,1,0,0,2
1,-0.468674,0,2,6,946771200000000000,2000,1,2,6,3,4,0,0,0,2,1
2,1.767960,0,0,8,946598400000000000,1999,12,31,4,4,3,0,1,0,2,2
3,-0.118771,1,3,NaN,946684800000000000,2000,1,1,5,7,0,1,0,1,2,3
4,0.630196,0,1,NaN,946598400000000000,1999,12,31,4,4,3,1,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-1.182318,-1,0,1,946684800000000000,2000,1,1,5,2,5,0,1,1,0,2
96,0.562761,0,0,NaN,946684800000000000,2000,1,1,5,7,0,0,0,3,1,2
97,-0.797270,0,1,6,946684800000000000,2000,1,1,5,3,4,0,0,0,2,1
98,0.502741,0,3,NaN,946598400000000000,1999,12,31,4,7,0,3,0,0,1,2


In [ ]:
df = pd.concat([dfx, dfy], axis=1)
predictor = TabularPredictor(label='label')
predictor.fit(df, hyperparameters={'GBM' : {}}, feature_generator=auto_ml_pipeline_feature_generator)

No path specified. Models will be saved in: "AutogluonModels/ag-20240921_063351"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       11.27 GB / 12.67 GB (88.9%)
Disk Space Avail:   71.17 GB / 112.64 GB (63.2%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : F

In [ ]:
print(len(set(dfx['B'])))

5


In [ ]:
dfx["B"] = dfx["B"].astype("category")
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
auto_ml_pipeline_feature_generator.fit_transform(X=dfx)

Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11415.61 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting DatetimeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['E']
			CountVectorizer fit with vocabulary size = 4
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types o

,A,B,D,E,C,C.year,C.month,C.day,C.dayofweek,E.char_count,E.symbol_ratio.,__nlp__.abc,__nlp__.ef,__nlp__.ghi,__nlp__.jkl,__nlp__._total_
0,-0.545774,1,3,NaN,946684800000000000,2000,1,1,5,4,3,2,1,0,0,2
1,-0.468674,1,2,6,946771200000000000,2000,1,2,6,3,4,0,0,0,2,1
2,1.767960,1,0,8,946598400000000000,1999,12,31,4,4,3,0,1,0,2,2
3,-0.118771,2,3,NaN,946684800000000000,2000,1,1,5,7,0,1,0,1,2,3
4,0.630196,1,1,NaN,946598400000000000,1999,12,31,4,4,3,1,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-1.182318,0,0,1,946684800000000000,2000,1,1,5,2,5,0,1,1,0,2
96,0.562761,1,0,NaN,946684800000000000,2000,1,1,5,7,0,0,0,3,1,2
97,-0.797270,1,1,6,946684800000000000,2000,1,1,5,3,4,0,0,0,2,1
98,0.502741,1,3,NaN,946598400000000000,1999,12,31,4,7,0,3,0,0,1,2


In [ ]:
dfx.iloc[0] = np.nan
dfx.head()

,A,B,C,D,E
0,NaN,NaN,NaT,NaN,NaN
1,-0.468674,0,2000-01-02,x,jkl d jkl d
2,1.767960,0,1999-12-31,v,jkl jkl ef d
3,-0.118771,1,2000-01-01,y,abc ghi jkl jkl
4,0.630196,0,1999-12-31,w,ef d abc jkl


In [ ]:
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
auto_ml_pipeline_feature_generator.fit_transform(X=dfx)

Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11415.88 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting DatetimeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['E']
			CountVectorizer fit with vocabulary size = 4
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types o

,A,B,D,E,C,C.year,C.month,C.day,C.dayofweek,E.char_count,E.word_count,E.symbol_ratio.,__nlp__.abc,__nlp__.ef,__nlp__.ghi,__nlp__.jkl,__nlp__._total_
0,NaN,NaN,NaN,NaN,946687418181818240,2000,1,1,5,0,0,0,0,0,0,0,0
1,-0.468674,1,2,6,946771200000000000,2000,1,2,6,4,1,5,0,0,0,2,1
2,1.767960,1,0,8,946598400000000000,1999,12,31,4,5,1,4,0,1,0,2,2
3,-0.118771,2,3,NaN,946684800000000000,2000,1,1,5,8,1,1,1,0,1,2,3
4,0.630196,1,1,NaN,946598400000000000,1999,12,31,4,5,1,4,1,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-1.182318,0,0,1,946684800000000000,2000,1,1,5,3,1,6,0,1,1,0,2
96,0.562761,1,0,NaN,946684800000000000,2000,1,1,5,8,1,1,0,0,3,1,2
97,-0.797270,1,1,6,946684800000000000,2000,1,1,5,4,1,5,0,0,0,2,1
98,0.502741,1,3,NaN,946598400000000000,1999,12,31,4,8,1,1,3,0,0,1,2


In [ ]:
from autogluon.features.generators import PipelineFeatureGenerator, CategoryFeatureGenerator, IdentityFeatureGenerator
from autogluon.common.features.types import R_INT, R_FLOAT
mypipeline = PipelineFeatureGenerator(
    generators = [[
        CategoryFeatureGenerator(maximum_num_cat=10),  # Overridden from default.
        IdentityFeatureGenerator(infer_features_in_args=dict(valid_raw_types=[R_INT, R_FLOAT])),
    ]]
)


In [ ]:
mypipeline.fit_transform(X=dfx)

Fitting PipelineFeatureGenerator...
	Available Memory:                    11366.94 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Unused Original Features (Count: 1): ['C']
		These features were not used to generate any of the output features. Add a feature generator compatible with these features to utilize them.
		Features can also be unused if they carry very little information, such as being categorical but having almost entirely uniqu

,B,D,E,A
0,NaN,NaN,NaN,NaN
1,1,2,6,-0.468674
2,1,0,8,1.767960
3,2,3,NaN,-0.118771
4,1,1,NaN,0.630196
...,...,...,...,...
95,0,0,1,-1.182318
96,1,0,NaN,0.562761
97,1,1,6,-0.797270
98,1,3,NaN,0.502741
